## Import packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from skimage import measure
from skimage.measure import regionprops, regionprops_table
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.preprocessing.image import load_img
from importlib import reload
import segmenteverygrain as seg
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
from tqdm import trange
%matplotlib qt

## Load models

In [2]:
model = seg.Unet()
model.compile(optimizer=Adam(), loss=seg.weighted_crossentropy, metrics=["accuracy"])
model.load_weights('./checkpoints/seg_model');

# the SAM model checkpoints can be downloaded from: https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
sam = sam_model_registry["default"](checkpoint="/Users/zoltan/Dropbox/Segmentation/sam_vit_h_4b8939.pth")

2024-03-28 10:29:09.554711: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-28 10:29:09.554894: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M2 Max


## Run segmentation

Grains are supposed to be well defined in the image; e.g., if a grain consists of only a few pixels, it is unlikely to be detected.

The segmentation can take a few minutes even for medium-sized images, so do not start with large images (downsample them if necessary). Images with ~2000 pixels along their largest dimension are a good start.

In [3]:
# replace this with the path to your image:
fname = '/Users/zoltan/Dropbox/Segmentation/torrey_pines_beach.jpeg'

big_im = np.array(load_img(fname))
big_im_pred = seg.predict_big_image(big_im, model, I=256)

# decreasing the 'dbs_max_dist' parameter results in more SAM prompts (and longer processing times):
labels, grains, coords = seg.label_grains(big_im, big_im_pred, dbs_max_dist=10.0) # Unet prediction

# SAM segmentation, using the point prompts from the Unet:
all_grains, labels, mask_all, grain_data, fig, ax = seg.sam_segmentation(sam, big_im, big_im_pred, 
                                                                         coords, labels, min_area=50.0)

  0%|                                                     | 0/7 [00:00<?, ?it/s]2024-03-28 10:29:23.257100: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-03-28 10:29:23.328440: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
100%|███████████████████████████████████████| 1457/1457 [01:15<00:00, 19.29it/s]
1454it [01:42, 14.19it/s]
100%|████████████████████████████████████████| 489/489 [00:01<00:00, 248.84it/s]


Use this figure to check the distribution of SAM prompts (= black dots):

In [5]:
plt.figure(figsize=(15,10))
plt.imshow(big_im)
plt.scatter(coords[:,0], coords[:,1], c='k')
plt.xticks([])
plt.yticks([]);

## Delete or merge grains in segmentation result
* click on the grain that you want to remove and press the 'x' key
* click on two grains that you want to merge and press the 'm' key (they have to be the last two grains you clicked on)
* press the 'g' key to hide the grain masks (so that you can see the original image better); press the 'g' key again to show the grain masks

In [6]:
grain_inds = []
cid1 = fig.canvas.mpl_connect('button_press_event', 
                              lambda event: seg.onclick2(event, all_grains, grain_inds, ax=ax))
cid2 = fig.canvas.mpl_connect('key_press_event', 
                              lambda event: seg.onpress2(event, all_grains, grain_inds, fig=fig, ax=ax))

Run this cell if you do not want to delete / merge existing grains anymore; it is a good idea to do this before moving on to the next step.

In [16]:
fig.canvas.mpl_disconnect(cid1)
fig.canvas.mpl_disconnect(cid2)

Use this function to update the 'all_grains' list after deleting and merging grains:

In [17]:
all_grains, labels, mask_all, fig, ax = seg.get_grains_from_patches(ax, big_im)

100%|███████████████████████████████████████| 1591/1591 [00:31<00:00, 51.23it/s]


Plot the updated set of grains:

In [75]:
fig, ax = plt.subplots(figsize=(15,10))
ax.imshow(big_im)
plt.xticks([])
plt.yticks([])
seg.plot_image_w_colorful_grains(big_im, all_grains, ax, cmap='Paired')
seg.plot_grain_axes_and_centroids(all_grains, labels, ax, linewidth=1, markersize=10)
plt.xlim([0, np.shape(big_im)[1]])
plt.ylim([np.shape(big_im)[0], 0]);

## Add new grains using the Segment Anything Model

* click on unsegmented grain that you want to add
* press the 'x' key if you want to delete the last grain you added
* press the 'm' key if you want to merge the last two grains that you added
* right click outside the grain (but inside the most recent mask) if you want to restrict the grain to a smaller mask - this adds a background prompt

In [76]:
predictor = SamPredictor(sam)
predictor.set_image(big_im) # this can take a while
coords = []
cid3 = fig.canvas.mpl_connect('button_press_event', lambda event: seg.onclick(event, ax, coords, big_im, predictor))
cid4 = fig.canvas.mpl_connect('key_press_event', lambda event: seg.onpress(event, ax, fig))

In [484]:
fig.canvas.mpl_disconnect(cid3)
fig.canvas.mpl_disconnect(cid4)

After you are done with the deletion / addition of grain masks, run this cell to generate an updated set of grains:

In [485]:
all_grains, labels, mask_all, fig, ax = seg.get_grains_from_patches(ax, big_im)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 443/443 [00:03<00:00, 121.37it/s]


## Get grain size distribution

Run this cell and then click (left mouse button) on one end of the scale bar in the image and click (right mouse button) on the other end of the scale bar:

In [20]:
cid5 = fig.canvas.mpl_connect('button_press_event', lambda event: seg.click_for_scale(event, ax))

number of pixels: 585.26


Use the length of the scale bar in pixels (it should be printed above) to get the scale of the image (in units / pixel):

In [21]:
n_of_units = 500 # centimeters in the case of 'IMG_5208_image.png'
units_per_pixel = n_of_units/585.26 # length of scale bar in pixels

In [27]:
n_of_units = 10 # centimeters in the case of 'IMG_5208_image.png'
units_per_pixel = n_of_units/492.06 # length of scale bar in pixels

In [22]:
props = regionprops_table(labels.astype('int'), intensity_image = big_im, properties =\
        ('label', 'area', 'centroid', 'major_axis_length', 'minor_axis_length', 
         'orientation', 'perimeter', 'max_intensity', 'mean_intensity', 'min_intensity'))
grain_data = pd.DataFrame(props)
grain_data['major_axis_length'] = grain_data['major_axis_length'].values*units_per_pixel
grain_data['minor_axis_length'] = grain_data['minor_axis_length'].values*units_per_pixel
grain_data['perimeter'] = grain_data['perimeter'].values*units_per_pixel
grain_data['area'] = grain_data['area'].values*units_per_pixel**2

In [23]:
grain_data.head()

,label,area,centroid-0,centroid-1,major_axis_length,minor_axis_length,orientation,perimeter,max_intensity-0,max_intensity-1,max_intensity-2,mean_intensity-0,mean_intensity-1,mean_intensity-2,min_intensity-0,min_intensity-1,min_intensity-2
0,1,1669.200428,854.374727,3469.557499,49.584006,43.227162,-0.226203,151.968570,255.0,255.0,255.0,191.974202,189.985571,186.794053,0.0,20.0,24.0
1,2,1030.568869,1192.178470,2176.718130,42.373409,31.535870,-1.176604,123.214808,255.0,255.0,255.0,190.112606,190.262748,187.290368,27.0,41.0,35.0
2,3,1998.369380,2255.628561,28.438276,72.259820,38.787062,0.833512,206.058811,255.0,255.0,255.0,177.666910,185.546749,187.480278,11.0,0.0,11.0
3,4,2541.388671,28.495118,1496.336588,74.802205,44.167385,-1.117318,206.058811,255.0,255.0,255.0,167.320505,169.190982,168.500287,0.0,0.0,15.0
4,5,1264.125554,685.515589,13.765012,57.766469,32.990824,0.156912,162.427717,255.0,255.0,255.0,144.542725,140.784642,134.379330,4.0,11.0,17.0


In [30]:
plt.figure()
plt.hist(grain_data['major_axis_length'], np.arange(0, 200, 5), alpha=0.5)
plt.hist(grain_data['minor_axis_length'], np.arange(0, 200, 5), alpha=0.5)

plt.xlabel('axis length (microns)')
plt.xlim(0, 200)
plt.ylabel('count');

2023-10-10 14:45:22.524 python[49126:39784776] +[CATransaction synchronize] called within transaction
2023-10-10 14:45:25.291 python[49126:39784776] +[CATransaction synchronize] called within transaction
2023-10-10 14:45:26.252 python[49126:39784776] +[CATransaction synchronize] called within transaction
2023-10-10 14:45:26.515 python[49126:39784776] +[CATransaction synchronize] called within transaction


## Save mask and grain labels to PNG files

In [486]:
dirname = '/Users/zoltan/Dropbox/Segmentation/images/'
# write grayscale mask to PNG file
cv2.imwrite(dirname + fname.split('/')[-1][:-4] + '_mask.png', mask_all)
# Save the image as a PNG file
cv2.imwrite(dirname + fname.split('/')[-1][:-4] + '_image.png', cv2.cvtColor(big_im, cv2.COLOR_BGR2RGB))

True